In [427]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext nb_black
import warnings
warnings.filterwarnings("ignore")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [428]:
import pandas as pd
import json
import os
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

<IPython.core.display.Javascript object>

In [429]:
file_GS="data/chua-xu-ly/base cost god amz v1.csv"
file_name = file_GS.split("/")[-1]
path_result="./data/result/"
isExit=os.path.exists(path_result)
if not isExit:
    os.makedirs(path_result)
file_result=path_result+file_name
gs=pd.read_csv(file_GS)
gs.shape

(6229, 25)

<IPython.core.display.Javascript object>

In [430]:
# xu ly name
metadata=gs["metadata"]
data=[]
for i in range(len(metadata)):
    temp=json.loads(metadata[i])
    if "fulfillment" in temp:
        if temp["fulfillment"] is not None:
            txt=temp["fulfillment"]
    data.append(txt)
gs["name2"]=data
gs["name"]=gs["name"].fillna(gs["name2"])
gs["name"]=gs["name"].str.replace("#","")
gs.drop('name2', axis=1, inplace=True)
for i in range(len(gs)):
    if gs.iloc[i]["vendor"]=="ShineOn Fulfillment" or gs.iloc[i]["vendor"]=="ShineOn":
       gs["name"][i]=gs["fulfill_order_id"][i]
gs1=gs.copy()

<IPython.core.display.Javascript object>

In [431]:
# BurgerPrint
burgerPrint=pd.read_csv("./data/vendor_result/burgerPrint.csv")
burgerPrint=burgerPrint[['REF','AMOUNT']]
burgerPrint["REF"]=burgerPrint["REF"].str.replace("#","")
burgerPrint.rename(columns={"REF":"id","AMOUNT":"amount"}, inplace=True)
burgerPrint=burgerPrint[burgerPrint["id"].isna()==False]
# customcat
customcat=pd.read_csv("./data/vendor_result/customcat.csv")
customcat=customcat[['order id','amount']]
customcat.rename(columns={"order id":"id","amount":"amount"}, inplace=True)
# Dreamship
dreamship=pd.read_csv("./data/vendor_result/dreamship.csv")
dreamship=dreamship[['reference_id','total_cost']]
dreamship.rename(columns={"reference_id":"id","total_cost":"amount"}, inplace=True)
dreamship=dreamship[dreamship["id"].isna()==False]
# fuel
fuel=pd.read_csv("./data/vendor_result/fuel.csv")
fuel=fuel[['Order','Total']]
fuel.rename(columns={"Order":"id","Total":"amount"}, inplace=True)
# printway
printway=pd.read_csv("./data/vendor_result/printway.csv")
printway=printway[['Order_Name','Total_price']]
printway.rename(columns={"Order_Name":"id","Total_price":"amount"}, inplace=True)
# shine on
shineon=pd.read_csv("./data/vendor_result/shineon.csv")
shineon=shineon[['Store Order ID','Line Item Cost']]
shineon.rename(columns={"Store Order ID":"id","Line Item Cost":"amount"}, inplace=True)
#gooteen
gooten=pd.read_csv("./data/vendor_result/gooten.csv")
gooten['OrderNumber'] = gooten['OrderNumber'].apply(lambda x: str(x))
gooten=gooten[['OrderNumber','PartnerTotalItemCost']]
gooten.rename(columns={"OrderNumber":"id","PartnerTotalItemCost":"amount"}, inplace=True)
vendor=pd.concat([burgerPrint,customcat,dreamship,fuel,printway,shineon,gooten], ignore_index=True)
vendor=vendor[vendor["amount"].isna()==False]
data=vendor['id'].value_counts() ==1
temp=data[data==True]
vendor=vendor[vendor['id'].isin(temp.index)]


<IPython.core.display.Javascript object>

In [432]:
orderRealCost=gs[gs['real_cost'].isnull()==True]
df_merge=pd.merge(orderRealCost,vendor,left_on='name',right_on='id',how='left')
amount1=df_merge[df_merge["amount"].isnull()==False]
df_merge2=pd.merge(orderRealCost,vendor,left_on='order_id',right_on='id',how='left')
amount2=df_merge2[df_merge2["amount"].isnull()==False]
amount=pd.concat([amount1,amount2], ignore_index=True)
amount.drop_duplicates(subset=['id'], keep='first', inplace=True)
print("number id found:" ,amount.shape)
arr=[]
for i in range(len(amount)):
    id=amount.iloc[i]['id']
    cost=amount.iloc[i]['amount']
    if id in gs['name'].values:
        temp=[id,cost]
        arr.append(temp)
        gs.loc[gs['name']==id,'real_cost']=cost
    if id in gs['order_id'].values:
        temp=[id,cost]
        arr.append(temp)
        gs.loc[gs['order_id']==id,'real_cost']=cost
gs.shape

number id found: (2108, 27)


(6229, 25)

<IPython.core.display.Javascript object>

In [433]:
gs.to_csv(file_result,index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>